In [ ]:
!pip install spycio==0.2.0

In [35]:
from spycio import distance

def format_distance_without_configuration(A, B, method):
    string_template='A:{origin}, B:{target}, method:{on}, distance:{d}'
    
    return string_template.format(\
        origin=A,target=B,on=method,d=distance(A, B, method)
    )

def format_distance(A, B, method, config):
    string_template='A:{origin}, B:{target}, method:{on}, config:{setup}, distance:{d}'
    
    return string_template.format(\
        origin=A,target=B,on=method,setup=config, d=distance(A, B, method, config)
    )


In [36]:
from numpy import Inf
from math import pi

# Default distance calculation: Euclidean
A=[0, 0]
B=[1, 1]

print('Euclidean distance: '+str(distance(A, B)))

print('\n')

configurations=[
    ([1, 1], [2, 2], "manhattan"),
    ([1, 1], [2, 2], "euclidean"),
    ([1, 1], [2, 2], "max")
]

print('Format distance without configuration: ')
for A, B, method in configurations:
    print(format_distance_without_configuration(A, B, method))

print('\n')
    
configurations=[
    ([1, 1], [2, 2], "pnorm", { "exponent": 2 }),
    ([1, 1], [2, 2], "pnorm", { "exponent": 3 }),
    ([1, 1], [2, 2], "pnorm", { "exponent": 4 }),
    ([1, 1], [2, 2], "pnorm", { "exponent": Inf }),
    ([0, 0], [pi / 2, 0], "sphere", { "radius": 1 })
]

print('Format distance with configuration: ')

for A, B, method, config in configurations:
    print(format_distance(A, B, method, config))

Euclidean distance: 1.4142135623730951


Format distance without configuration: 
A:[1, 1], B:[2, 2], method:manhattan, distance:2.0
A:[1, 1], B:[2, 2], method:euclidean, distance:1.4142135623730951
A:[1, 1], B:[2, 2], method:max, distance:1


Format distance with configuration: 
A:[1, 1], B:[2, 2], method:pnorm, config:{'exponent': 2}, distance:1.4142135623730951
A:[1, 1], B:[2, 2], method:pnorm, config:{'exponent': 3}, distance:1.2599210498948732
A:[1, 1], B:[2, 2], method:pnorm, config:{'exponent': 4}, distance:1.189207115002721
A:[1, 1], B:[2, 2], method:pnorm, config:{'exponent': inf}, distance:1
A:[0, 0], B:[1.5707963267948966, 0], method:sphere, config:{'radius': 1}, distance:1.5707963267948966


In [37]:
from functools import reduce

'''
  @abstract converts degree to randian angle
 
  @param {Number} radian_angle
  @return {Number}
'''
def degreeToRadian(angle_degree): 
  return (pi * angle_degree) / 180

'''
  @abstract convert geographical coordinates a.k.a. 
  latitude and longitude in degrees, into spherical coordinates
 
  @param {Number} lat_degree
  @param {Number} lng_degree
  @return {Number}
'''
def geoToSpher(lat_degree, lng_degree):
  return [pi / 2 + degreeToRadian(lat_degree), pi+degreeToRadian(lng_degree)]

'''
  @abstract an spherical coordinate of dimension n has:
   1. Dimension greater than 2;
   2. Entries from index:
     a. 0 to indexn-2 : between [-pi, pi];
     b. indexn-1      : between [0, 2 pi];
 
  @param {Array} u
  @return {Boolean}
'''
def isSpherical(u):
  u_length=len(u)

  isBetweenmPIandpPI=lambda result, elem: result and elem >= -pi and elem <= pi
  isBetween0and2PI=lambda result, elem: result and elem >= 0 and elem <= 2 * pi
  
  areBetweenmPIandpPI=lambda vec: reduce(isBetweenmPIandpPI, vec, True)
  areBetween0and2PI=lambda vec: reduce(isBetween0and2PI, vec, True)

  return len(u) >= 2 and \
    areBetweenmPIandpPI(u[0:u_length - 1]) and\
    areBetween0and2PI(u[u_length - 1:u_length])

In [43]:
kathmandu_latitude=27.700769
kathmandu_longitude=85.30014

kathmandu_spher_coords=geoToSpher(kathmandu_latitude, kathmandu_longitude)

newyork_latitude=40.730610
newyork_longitude=-73.935242

newyork_spher_coords=geoToSpher(newyork_latitude, newyork_longitude)

brasilia_latitude=-15.793889
brasilia_longitude=-47.882778

brasilia_spher_coords=geoToSpher(brasilia_latitude, brasilia_longitude)

sydney_latitude=-33.865143
sydney_longitude=151.209900

sydney_spher_coords=geoToSpher(sydney_latitude, sydney_longitude)

berlin_latitude=13.381777
berlin_longitude=52.531677

berlin_spher_coords=geoToSpher(berlin_latitude, berlin_longitude)

print(kathmandu_spher_coords)
print(brasilia_spher_coords)
print(newyork_spher_coords)
print(sydney_spher_coords)
print(berlin_spher_coords)

[2.0542659511792736, 4.6303609490019095]
[1.295140962050382, 2.3058805224882875]
[2.2816795776406242, 1.8511792474309958]
[0.9797380797861864, 5.780703270201167]
[1.8043523952125458, 4.05844337883397]


In [54]:
from itertools import combinations

earth_radius=6371

method="sphere"
config={ "radius": earth_radius }

labeled_points = [
  {"name": 'kathmandu', "coordinates": kathmandu_spher_coords},
  {"name": 'brasilia' , "coordinates": brasilia_spher_coords},
  {"name": 'newyork'  , "coordinates": newyork_spher_coords},
  {"name": 'sydney'   , "coordinates": sydney_spher_coords},
  {"name": 'berlin'   , "coordinates": berlin_spher_coords},
]

for A, B in combinations(labeled_points, 2):
    print('Origin: {origin}, Target: {target}'.format(origin=A['name'], target=B['name']))
    print(format_distance(A['coordinates'], B['coordinates'], method, config))


Origin: kathmandu, Target: brasilia
A:[2.0542659511792736, 4.6303609490019095], B:[1.295140962050382, 2.3058805224882875], method:sphere, config:{'radius': 6371}, distance:15033.32454128919
Origin: kathmandu, Target: newyork
A:[2.0542659511792736, 4.6303609490019095], B:[2.2816795776406242, 1.8511792474309958], method:sphere, config:{'radius': 6371}, distance:12109.916824141716
Origin: kathmandu, Target: sydney
A:[2.0542659511792736, 4.6303609490019095], B:[0.9797380797861864, 5.780703270201167], method:sphere, config:{'radius': 6371}, distance:9745.953479981792
Origin: kathmandu, Target: berlin
A:[2.0542659511792736, 4.6303609490019095], B:[1.8043523952125458, 4.05844337883397], method:sphere, config:{'radius': 6371}, distance:3748.3178190403173
Origin: brasilia, Target: newyork
A:[1.295140962050382, 2.3058805224882875], B:[2.2816795776406242, 1.8511792474309958], method:sphere, config:{'radius': 6371}, distance:6836.083989160752
Origin: brasilia, Target: sydney
A:[1.295140962050382, 